# Resnet Finetuned on MSCOCO

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_graph import simple_layer, variable_summaries
from utils.data import init_model_logging
from utils.data import Dataset

import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.contrib import slim

## Load data

In [ ]:
class data:
    train = Dataset('/data/mscoco/train_data.hdf', '/data/mscoco/train_imgs/', one_hot=True, norm=False)
    validation = Dataset('/data/mscoco/valid_data.hdf', '/data/mscoco/valid_imgs/', one_hot=True, norm=False)
    
class_id2class_name_mapping = {
    0: 'cow',
    1: 'sheep',
    2: 'giraffe',
    3: 'horse',
    4: 'bird',
    5: 'cat',
    6: 'dog',
    7: 'elephant',
    8: 'bear'}

## Build Resnet Graph

In [ ]:
resnet_model_ckpt = '/data/checkpoints/resnet_v1_50.ckpt'
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('resnet_inputs'):
        images = tf.placeholder(dtype=tf.float32, shape=[None, 224 * 224 * 3], name='images')
        labels = tf.placeholder(tf.float32, shape=(None, 9), name='labels')
        is_training = tf.placeholder(tf.bool, shape=(), name='training_flag')

    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 224, 224, 3])        

    #########################
    # Resnet part of model. #
    #########################
        
    # Run resnet in mode, where it returns embedding insted of prediction itself.
    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        resnet_tensors = resnet_v1.resnet_v1_50(
            images_reshaped, is_training=is_training, num_classes=None,
            global_pool=True, scope='resnet_v1_50')
        
    # Here we get embedding vector from pretrained model. 
    # This is point where to join our custom network.
    resnet_feature_vector, resnet_endpoints = resnet_tensors

    with tf.name_scope('resnet_feature_vector'):
        resnet_feature_vector = tf.reshape(resnet_feature_vector, shape=(-1, 2048))

    ########################
    # Our Custom Netowork. #
    ########################
        
    with tf.variable_scope('layer1'):
        prediction_raw = resnet_feature_vector
        prediction_raw = simple_layer('layer1', prediction_raw, shape=[2048, 1024], activation='relu')
        
    with tf.variable_scope('layer2'):
        prediction_raw = simple_layer('layer2', prediction_raw, shape=[1024, 9], activation='linear')

    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(prediction_raw)

    #############################################
    # Operations for resnet part initialization #
    #############################################
            
    init_resnet = slim.assign_from_checkpoint_fn(resnet_model_ckpt, slim.get_model_variables('resnet_v1_50'))
    initialize_vars = tf.group(
        tf.global_variables_initializer(),
        tf.local_variables_initializer())
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/resnet/'
exp_name = 'experiment_finetune_two_optimizers'
exp_name = 'experiment_finetune_one_optimizer'

logging_meta = init_model_logging(base_dir, exp_name, graph=graph, remove_existing=False)

## Load and Run Pretrained Resnet

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']+'-1900'

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    is_training: False}

session = tf.Session(graph=graph, config=config)
logging_meta['saver'].restore(session, model_path)
_prediction, = session.run([prediction], validation_feed_dict)

In [ ]:
session.close()

## Results evaluation

In [ ]:
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df, use_top3=False)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
plot_coocurance_matrix(df, use_log=False)

In [ ]:
fp = get_false_positives(df, 'horse')

In [ ]:
plot_examples(fp, [224, 224 , 3])